In [1]:
import gym
from baselines import deepq
import tensorflow as tf

def callback(lcl, _glb):
    # stop training if reward exceeds 199
    is_solved = lcl['t'] > 100 and sum(lcl['episode_rewards'][-101:-1]) / 100 >= 199
    return is_solved

Logging to /tmp/openai-2018-09-14-15-56-56-153504


In [2]:
def train_cartpole():
    env = gym.make("CartPole-v0")
    act = deepq.learn(
        env,
        network='mlp',
        lr=1e-3,
        total_timesteps=10000,
        buffer_size=50000,
        exploration_fraction=0.1,
        exploration_final_eps=0.02,
        print_freq=100,
        callback=callback
    )
    print("Saving model to cartpole_model.pkl")   
    return act

In [3]:
def evaluate_cartpole(act):
    env = gym.make("CartPole-v0")    
    obs, done = env.reset(), False
    episode_rew = 0
    
    while not done:
        # env.render()
        obs, rew, done, _ = env.step(act(obs[None])[0])
        episode_rew += rew
    print("Episode reward", episode_rew)

In [4]:
def train_mountaincar():
    env = gym.make("MountainCar-v0")
    
    act = deepq.learn(
        env,
        network='mlp',        
        lr=1e-3,
        total_timesteps=10000,
        buffer_size=50000,
        exploration_fraction=0.1,
        exploration_final_eps=0.1,
        print_freq=100,
        param_noise=True,
        callback=None
    )
    print("Saving model to mountaincar_model.pkl")
    return act      

In [5]:
def evaluate_mountaincar(act):
    env = gym.make("MountainCar-v0")    
    
    obs, done = env.reset(), False
    episode_rew = 0
    while not done:
        # env.render()
        obs, rew, done, _ = env.step(act(obs[None])[0])
        episode_rew += rew
    print("Episode reward", episode_rew)    
    env.close()

In [6]:
# Train CartPole
with tf.variable_scope('cartpole'):
    cartpole_action = train_cartpole()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/yhbyhb/anaconda3/envs/deeprl/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/yhbyhb/anaconda3/envs/deeprl/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 100      |
| mean 100 episode reward | 16.8     |
| steps                   | 1665     |
--------------------------------------
Saving model to cartpole_model.pkl


In [7]:
# Evaluate CartPole
evaluate_cartpole(cartpole_action)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode reward 200.0


In [8]:
# Train MountainCar
with tf.variable_scope('mtncar'):
    mtncar_action = train_mountaincar()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/yhbyhb/anaconda3/envs/deeprl/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/yhbyhb/anaconda3/envs/deeprl/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Saving model to mountaincar_model.pkl


In [9]:
# Evaluate MountainCar
evaluate_mountaincar(mtncar_action)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode reward -200.0
